In [1]:
%matplotlib inline
import pandas as pd
from pandas import set_option
#Limit dataframe exibition to 8 rows
set_option("display.max_rows", 8)
import matplotlib.pyplot as plt
from pylab import plot
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

In [2]:
customers = pd.read_csv('orders_customers/customers.txt', sep='\t', lineterminator='\r')

/home/guilherme/ENV/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2705: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
orders = pd.read_csv('orders_customers/orders.txt', sep='\t', lineterminator='\r')

In [4]:
orders.ix[orders['customerid']==0]

,orderid,customerid,campaignid,orderdate,city,state,zipcode,paymenttype,totalprice,numorderlines,numunits
65,1002444,0.0,2204.0,2011-01-12 00:00:00,AUCKLAND,NaN,0,VI,10.00,1.0,1.0
188,1004051,0.0,2173.0,2010-11-23 00:00:00,TORONTO,ON,M5R3R,VI,50.00,1.0,5.0
207,1003454,0.0,2173.0,2011-01-13 00:00:00,CZECH REPUBLIC,NaN,NaN,MC,21.00,1.0,1.0
337,1003587,0.0,2173.0,2011-01-13 00:00:00,VANCOUVER,BC,V6N2T,MC,10.00,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
192874,\n1641185,0.0,2141.0,2016-09-06 00:00:00,MONTREAL,PQ,H2Y4A,MC,34.10,3.0,3.0
192881,\n1641131,0.0,2236.0,2016-09-16 00:00:00,ST HELENS,NaN,WA92a,VI,89.95,1.0,1.0
192929,\n1642126,0.0,2236.0,2016-09-18 00:00:00,WESTMOUNT,PQ,H3Y1X,AE,54.95,1.0,1.0
192935,\n1642132,0.0,2236.0,2016-09-16 00:00:00,OTTAWA,ON,K2J5E,AE,17.00,1.0,1.0


In [5]:
customers.ix[customers['customerid']==0]

,customerid,householdid,gender,firstname


### As we can see above, customerid==0 is not tracked so we will drop them

In [6]:
orders = orders.ix[orders['customerid']!=0]

In [7]:
orders

,orderid,customerid,campaignid,orderdate,city,state,zipcode,paymenttype,totalprice,numorderlines,numunits
0,1002854,45978.0,2141.0,2009-10-13 00:00:00,NEWTON,MA,02459,VI,190.00,3.0,3.0
1,1002855,125381.0,2173.0,2009-10-13 00:00:00,NEW ROCHELLE,NY,10804,VI,10.00,1.0,1.0
2,1002856,103122.0,2141.0,2011-06-02 00:00:00,MIAMI,FL,33137,AE,35.22,2.0,2.0
3,1002857,130980.0,2173.0,2009-10-14 00:00:00,E RUTHERFORD,NJ,07073,AE,10.00,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
192980,\n1643149,188938.0,2236.0,2016-09-18 00:00:00,HOUSTON,TX,77070,VI,16.95,1.0,1.0
192981,\n1643150,189517.0,2236.0,2016-09-18 00:00:00,PITTSBURGH,PA,15234,MC,22.95,1.0,1.0
192982,\n1643157,167434.0,2141.0,2016-09-20 00:00:00,NEW YORK,NY,10027,MC,49.45,1.0,1.0
192983,\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
orders.reset_index(drop=True, inplace=True)

In [9]:
orders.shape[0]

189560

In [10]:
len(orders['customerid'].unique())

189560

### As we can see above, the orders table has only 1 record for each customer

In [11]:
orders['orderdate'] = pd.to_datetime(orders['orderdate'])

In [12]:
orders.ix[orders['orderdate']>'2016-07-17'].shape[0]

2668

In [13]:
today = orders['orderdate'].max()

### As we have some order dates greater than today (2016-08-17), we will assume that the greatest order date is today's date

In [14]:
rfmTable = orders.groupby(by='customerid').agg({ 'orderdate' : lambda x: (today - x.max()).days, \
                                               'orderid': lambda x: len(x), \
                                               'totalprice': lambda x: x.sum()})

In [15]:
rfmTable['orderdate'] = rfmTable['orderdate'].astype(int)
rfmTable.rename(columns={'orderdate': 'R', 'orderid': 'F', 'totalprice': 'M'}, inplace=True)

In [16]:
rfmTable.sort_values(by=['R','F','M'], ascending=True, inplace=True)
rfmTable.reset_index(inplace=True)

In [17]:
# Calculate the size of the quartile
quartile = rfmTable.shape[0]//4

In [18]:
R1, R2, R3, R4 = rfmTable.ix[:quartile, :], rfmTable.ix[(quartile+1):(quartile*2), :], \
rfmTable.ix[((quartile*2)+1):(quartile*3), :], rfmTable.ix[((quartile*3)+1):, :]

In [19]:
rfmTable.sort_values(by=['M'], ascending=False, inplace=True)
rfmTable.reset_index(inplace=True)

In [20]:
M1, M2, M3, M4 = rfmTable.ix[:quartile, :], rfmTable.ix[(quartile+1):(quartile*2), :], \
rfmTable.ix[((quartile*2)+1):(quartile*3), :], rfmTable.ix[((quartile*3)+1):, :]

#### As we have only one order per customer, we can't procede with the most complex and interesting analysis of RFM, but we still can identify R1 as the customers who did the most recent orders. Also M1 tell us who spent more money. Unfortunately we can't say who are our best or more loyal customers. We can't sey either who are our new spenders, loyal Joes, and lot's of other categories that are possible to identify using RFM